<a href="https://colab.research.google.com/github/fatin-fiqirs/test/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import datetime
class EIADataFetcher:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.eia.gov/v2"

    def get_electricity_data(self, route):
        """
        Get electricity data from EIA API
        """
        url = f"{self.base_url}/{route}?api_key={self.api_key}"
        response = requests.get(url)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None

    def get_wholesale_prices(self):
        """
        Get wholesale electricity prices
        """
        route = "electricity/rto/region-data"
        return self.get_electricity_data(route)

# Example usage
def main():
    api_key = "0uJhb753HljL7ey0rVz58sfzHdnR2l0KjSQqcdrZ"  # Replace with your API key
    fetcher = EIADataFetcher(api_key)

    # Get wholesale prices
    data = fetcher.get_wholesale_prices()

    if data:
        # Convert to DataFrame for easier analysis
        df = pd.DataFrame(data['response']['data'])
        print(df.head())

        # Save to CSV
        df.to_csv('electricity_prices.csv', index=False)
        print("Data saved to electricity_prices.csv")

if __name__ == "__main__":
    main()

                                                                value
aggregation-method                                                SUM
alias               Demand, Demand Forecast, Net Generation, or In...
units                                                   megawatthours
Data saved to electricity_prices.csv


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

class EIADataFetcher:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.eia.gov/v2"

    def get_wholesale_prices(self):
        """
        Get wholesale electricity prices using the series endpoint
        """
        # Series IDs for wholesale prices
        # CAISO - California
        # ERCOT - Texas
        series_ids = {
            'CAISO': 'EBA.CAISO-ALL.NG.WAP.H',  # California wholesale price
            'ERCOT': 'EBA.ERCO-ALL.NG.WAP.H'    # Texas wholesale price
        }

        dfs = []
        for market, series_id in series_ids.items():
            endpoint = f"/electricity/rto/region-data/data"
            params = {
                "api_key": self.api_key,
                "data[]": ["value"],
                "facets[respondent][]": market,
                "frequency": "hourly",
                "start": (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%dT%H"),
                "end": datetime.now().strftime("%Y-%m-%dT%H"),
                "sort[0][column]": "period",
                "sort[0][direction]": "asc"
            }

            try:
                response = requests.get(f"{self.base_url}{endpoint}", params=params)
                print(f"URL requested: {response.url}")  # Print URL for debugging

                if response.status_code == 200:
                    data = response.json()
                    df = pd.DataFrame(data['response']['data'])
                    df['market'] = market
                    dfs.append(df)
                else:
                    print(f"Error fetching {market} data: Status code {response.status_code}")
                    print(f"Response: {response.text}")

            except Exception as e:
                print(f"Error occurred while fetching {market} data: {str(e)}")

        if not dfs:
            return None

        # Combine all data
        combined_df = pd.concat(dfs, ignore_index=True)

        # Process the dataframe
        combined_df['period'] = pd.to_datetime(combined_df['period'])
        processed_df = combined_df.pivot(index='period',
                                       columns='market',
                                       values='value').reset_index()

        # Add Atlanta (constant price as in paper)
        processed_df['Atlanta_GA'] = 55.30

        # Rename columns
        processed_df = processed_df.rename(columns={
            'period': 'Timestamp',
            'CAISO': 'Mountain_View_CA',
            'ERCOT': 'Houston_TX'
        })

        return processed_df

    def save_data(self):
        """
        Save the electricity price data
        """
        data = self.get_wholesale_prices()
        if data is not None:
            # Save to CSV
            data.to_csv('electricity_prices.csv', index=False)
            print("\nData saved to electricity_prices.csv")
            print("\nFirst few rows of the data:")
            print(data.head())

            # Display data info
            print("\nDataset information:")
            print(data.info())
        else:
            print("No data was retrieved")

def main():
    # Get API key from user
    api_key = input("Please enter your EIA API key: ")

    # Initialize fetcher and save data
    fetcher = EIADataFetcher(api_key)
    fetcher.save_data()

if __name__ == "__main__":
    main()

Please enter your EIA API key: 0uJhb753HljL7ey0rVz58sfzHdnR2l0KjSQqcdrZ
URL requested: https://api.eia.gov/v2/electricity/rto/region-data/data?api_key=0uJhb753HljL7ey0rVz58sfzHdnR2l0KjSQqcdrZ&data%5B%5D=value&facets%5Brespondent%5D%5B%5D=CAISO&frequency=hourly&start=2025-01-08T04&end=2025-01-15T04&sort%5B0%5D%5Bcolumn%5D=period&sort%5B0%5D%5Bdirection%5D=asc
URL requested: https://api.eia.gov/v2/electricity/rto/region-data/data?api_key=0uJhb753HljL7ey0rVz58sfzHdnR2l0KjSQqcdrZ&data%5B%5D=value&facets%5Brespondent%5D%5B%5D=ERCOT&frequency=hourly&start=2025-01-08T04&end=2025-01-15T04&sort%5B0%5D%5Bcolumn%5D=period&sort%5B0%5D%5Bdirection%5D=asc


KeyError: 'period'